# การแทนค่าแบบจำลอง (LABไม่บังคับ)
<figure>
 <img src="./images/C1_W1_L3_S1_Lecture_b.png"   style="width:600px;height:200px;">
</figure>

## เป้าหมาย
ในห้องปฏิบัติการนี้ คุณจะ:
- เรียนรู้การนำโมเดล $f_{w,b}$ ไปใช้ในการถดถอยเชิงเส้นที่มีตัวแปรเดียว


## สัญลักษณ์
นี่คือสรุปสัญลักษณ์บางอย่างที่คุณจะพบ 

- | $a$ | scalar, non bold                                                      ||
- | $\mathbf{a}$ | vector, bold                                                      ||
- | **Regression** |         |    |     |
- |  $\mathbf{x}$ | คุณลักษณะตัวอย่าง: ขนาด (1000 ตารางฟุต)  | `x_train` |   
- |  $\mathbf{y}$  | ชุดตัวอย่างการฝึก: เป้าหมาย (ในห้องปฏิบัติการนี้ ราคา (1000 ดอลลาร์))  | `y_train` 
- |  $x^{(i)}$, $y^{(i)}$ | $i_{th}$ ตัวอย่างการฝึก | `x_i`, `y_i`|
- | m | จำนวนตัวอย่างการฝึก | `m`|
- |  $w$  |  parameter: weight                                 | `w`    |
- |  $b$           |  parameter: bias                                           | `b`    |     
- | $f_{w,b}(x^{(i)})$ | ผลการประเมินโมเดลที่ $x^{(i)}$ ถูกกำหนดโดย $w,b$: $f_{w,b}(x^{(i)}) = wx^{(i)}+b$  | `f_wb` | 


## เครื่องมือ
ในห้องปฏิบัติการนี้ คุณจะใช้:

- NumPy ซึ่งเป็นไลบรารีที่ได้รับความนิยมสำหรับการคำนวณทางวิทยาศาสตร์
- Matplotlib ซึ่งเป็นไลบรารีที่ได้รับความนิยมสำหรับการพล็อตข้อมูล

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle'
response = requests.get(url)

with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')

# Problem Statement
<img align="left" src="./images/C1_W1_L3_S1_trainingdata_v1.png"    style=" width:380px; padding: 10px;  " /> 

การทำนายราคาบ้าน
เช่นเดียวกับการบรรยาย คุณจะใช้ตัวอย่างการทำนายราคาบ้านเป็นแรงจูงใจ

ห้องปฏิบัติการนี้จะใช้ชุดข้อมูลง่ายๆ ที่มีเพียงสองจุดข้อมูล - บ้านขนาด 1000 ตารางฟุต (sqft) ขายได้ 300,000 ดอลลาร์ และบ้านขนาด 2000 ตารางฟุต ขายได้ 500,000 ดอลลาร์ จุดข้อมูลสองจุดนี้จะประกอบเป็น ชุดข้อมูลหรือชุดฝึกอบรม ของเรา ในห้องปฏิบัติการนี้ หน่วยของขนาดคือ 1000 ตารางฟุต และหน่วยของราคาคือ 1000 ดอลลาร์

| Size (1000 sqft)     | Price (1000s of dollars) |
| -------------------| ------------------------ |
| 1.0               | 300                      |
| 2.0               | 500                      |

คุณต้องการปรับแบบจำลองการถดถอยเชิงเส้น (แสดงด้วยเส้นตรงสีน้ำเงินด้านบน) ผ่านสองจุดนี้ เพื่อที่คุณจะสามารถทำนายราคาบ้านอื่น ๆ - เช่น บ้านที่มีพื้นที่ 1200 ตารางฟุต


โปรดรันเซลล์โค้ดต่อไปนี้เพื่อสร้างตัวแปร `x_train` และ `y_train` ของคุณ ข้อมูลถูกเก็บในอาร์เรย์ NumPy แบบหนึ่งมิติ

In [ ]:
# x_train is the input variable (size in 1000 square feet)
# y_train is the target (price in 1000s of dollars)
x_train = np.array([1.0, 2.0])
y_train = np.array([300.0, 500.0])
print(f"x_train = {x_train}")
print(f"y_train = {y_train}")



>**หมายเหตุ**: เกี่ยวกับการใช้ f-string ใน Pythonหลักสูตรนี้จะใช้รูปแบบการแสดงผล "f-string" ของ Python บ่อยครั้ง เมื่อทำการพิมพ์ เนื้อหาที่อยู่ภายในวงเล็บปีกกาจะถูกประเมินในขณะที่สร้างผลลัพธ์ออกมา



### จำนวนตัวอย่างการฝึก` m`
คุณจะใช้ m เพื่อแทนจำนวนตัวอย่างการฝึก อาร์เรย์ Numpy มีพารามิเตอร์ `.shape` `x_train.shape` จะคืนค่าเป็น tuple ของ Python ที่มีรายการสำหรับแต่ละมิติ `x_train.shape[0]` คือความยาวของอาร์เรย์และจำนวนตัวอย่างดังแสดงด้านล่าง

In [ ]:
# m is the number of training examples
print(f"x_train.shape: {x_train.shape}")
m = x_train.shape[0]
print(f"Number of training examples is: {m}")

One can also use the Python `len()` function as shown below.

In [ ]:
# m is the number of training examples
m = len(x_train)
print(f"Number of training examples is: {m}")

### ชุดตัวอย่างการฝึก `x_i, y_i`

คุณจะใช้ ($x^{(i)}$, $y^{(i)}$) ในการบอก $i^{th}$ ของชุดในการฝึก เนื่องจาก Python มีการนับเริ่มต้นที่ 0, ($x^{(0)}$, $y^{(0)}$) คือ (1.0, 300.0) และ ($x^{(1)}$, $y^{(1)}$) คือ (2.0, 500.0). 

การเข้าถึงค่าในอาร์เรย์ Numpy
เพื่อเข้าถึงค่าในอาร์เรย์ Numpy เราจะใช้อินเด็กซ์ (index) ที่ต้องการกับอาร์เรย์นั้น ตัวอย่างเช่น ไวยากรณ์ในการเข้าถึงค่าที่ตำแหน่งศูนย์ของ `x_train` คือ `x_train[0]`.
รันบล็อกโค้ดต่อไปนี้เพื่อรับตัวอย่างการฝึกอบรมที่ $i^{th}$ training example.


In [ ]:
i = 0 # Change this to 1 to see (x^1, y^1)

x_i = x_train[i]
y_i = y_train[i]
print(f"(x^({i}), y^({i})) = ({x_i}, {y_i})")

### การพล็อตข้อมูล

การพล็อตจุดสองจุดด้วย scatter() ใน `matplotlib`
- คุณสามารถพล็อตจุดสองจุดนี้โดยใช้ฟังก์ชัน scatter() ในไลบรารี `matplotlib` ตามที่แสดงในเซลล์ด้านล่าง:

อาร์กิวเมนต์ฟังก์ชัน marker และ c แสดงจุดเป็นเครื่องหมายกากบาทสีแดง (ค่าเริ่มต้นคือจุดสีน้ำเงิน)
คุณสามารถใช้ฟังก์ชันอื่น ๆ ในไลบรารี `matplotlib` เพื่อตั้งชื่อและป้ายกำกับเพื่อแสดง

In [ ]:
# Plot the data points
plt.scatter(x_train, y_train, marker='x', c='r')
# Set the title
plt.title("Housing Prices")
# Set the y-axis label
plt.ylabel('Price (in 1000s of dollars)')
# Set the x-axis label
plt.xlabel('Size (1000 sqft)')
plt.show()

## Model function

<img align="left" src="./images/C1_W1_L3_S1_model_v1.png"     style=" width:380px; padding: 10px; " > ฟังก์ชันแบบจำลองสำหรับการถดถอยเชิงเส้น
ตามที่อธิบายในบรรยาย ฟังก์ชันแบบจำลองสำหรับการถดถอยเชิงเส้น (ซึ่งเป็นฟังก์ชันที่จับคู่จาก `x` ไปยัง `y`) ถูกแสดงเป็น

$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$

สูตรด้านบนคือวิธีการแทนเส้นตรง ค่าต่างๆ ของ `w` และ `b` จะให้เส้นตรงที่แตกต่างกันบนพล็อต <br/> <br/> <br/> <br/> <br/> 

มาลองเข้าใจแนวคิดนี้ผ่านบล็อกโค้ดด้านล่างกันเถอะ เริ่มจาก $w = 100$ และ $b = 100$. 

**หมายเหตุ: คุณสามารถกลับมาที่เซลล์นี้เพื่อปรับพารามิเตอร์ w และ b ของโมเดลได้**

In [ ]:
w = 100
b = 100
print(f"w: {w}")
print(f"b: {b}")

ตอนนี้ มาคำนวณค่าของ $f_{w,b}(x^{(i)})$ สำหรับสองจุดข้อมูลของคุณ คุณสามารถเขียนค่านี้สำหรับแต่ละจุดข้อมูลอย่างชัดเจนได้ดังนี้ -

ให้ $x^{(0)}$, `f_wb = w * x[0] + b`

ให้ $x^{(1)}$, `f_wb = w * x[1] + b`

สำหรับชุดข้อมูลจำนวนมาก การคำนวณแบบนี้จะยากลำบากและเกิดการซ้ำ ดังนั้น คุณสามารถคำนวณผลลัพธ์ของฟังก์ชันภายในลูป `for` ตามที่แสดงในฟังก์ชัน `compute_model_output` ด้านล่าง

>**หมายเหตุ**: คำอธิบายอาร์กิวเมนต์ `(ndarray (m,))` อธิบายอาร์เรย์ n-dimensional ของ NumPy ที่มีรูปร่าง (m,)  (scalar) อธิบายอาร์กิวเมนต์ที่ไม่มีมิติ เพียงแค่ขนาด
>**หมายเหตุ**: `np.zero(n)` จะส่งคืนอาร์เรย์ numpy หนึ่งมิติที่มีรายการ n รายการ


In [ ]:
def compute_model_output(x, w, b):
    """
    Computes the prediction of a linear model
    Args:
      x (ndarray (m,)): Data, m examples 
      w,b (scalar)    : model parameters  
    Returns
      f_wb (ndarray (m,)): model prediction
    """
    m = x.shape[0]
    f_wb = np.zeros(m)
    for i in range(m):
        f_wb[i] = w * x[i] + b
        
    return f_wb

ตอนนี้เราจะเรียกใช้ฟังก์ชัน `compute_model_output` และพล็อตผลลัพธ์..









In [ ]:
tmp_f_wb = compute_model_output(x_train, w, b,)

# Plot our model prediction
plt.plot(x_train, tmp_f_wb, c='b',label='Our Prediction')

# Plot the data points
plt.scatter(x_train, y_train, marker='x', c='r',label='Actual Values')

# Set the title
plt.title("Housing Prices")
# Set the y-axis label
plt.ylabel('Price (in 1000s of dollars)')
# Set the x-axis label
plt.xlabel('Size (1000 sqft)')
plt.legend()
plt.show()

ดังที่คุณเห็น การตั้งค่า $w=100$ และ $b=100$ ไม่ได้ส่งผลให้เกิดเส้นที่เข้ากับข้อมูลของเรา

### ความท้าทาย
ลองทดลองกับค่าต่างๆ ของ w และ b ค่าควรจะเป็นอย่างไรสำหรับเส้นที่เข้ากับข้อมูลของเรา?

### เคล็ดลับ:
คุณสามารถใช้เมาส์ของคุณคลิกที่ "คำแนะนำ" สีเขียวด้านล่างเพื่อเปิดเผยคำแนะนำบางอย่างสำหรับการเลือก b และ w









<details>
<summary>
    <font size='3', color='darkgreen'><b>Hints</b></font>
</summary>
    <p>
    <ul>
        <li>Try $w = 200$ and $b = 100$ </li>
    </ul>
    </p>

### การทำนาย
ตอนนี้เรามีโมเดลแล้ว เราสามารถใช้โมเดลนี้เพื่อทำนายค่าเดิมของเราได้ มาทำนายราคาของบ้านที่มีพื้นที่ 1200 ตารางฟุต เนื่องจากหน่วยของ  $x$  เป็น 1000 ตารางฟุต  $x$  จึงเท่ากับ 1.2

In [ ]:
w = 200                         
b = 100    
x_i = 1.2
cost_1200sqft = w * x_i + b    

print(f"${cost_1200sqft:.0f} thousand dollars")

### ขอแสดงความยินดี!
ในห้องปฏิบัติการนี้ คุณได้เรียนรู้:
การถดถอยเชิงเส้นสร้างแบบจำลองที่สร้างความสัมพันธ์ระหว่างคุณลักษณะและเป้าหมาย
- ในตัวอย่างข้างต้น คุณลักษณะคือขนาดบ้าน และเป้าหมายคือราคาบ้าน
- สำหรับการถดถอยเชิงเส้นอย่างง่าย แบบจำลองจะมีพารามิเตอร์สองตัวคือ w และ b ซึ่งค่าของพารามิเตอร์เหล่านี้จะถูก 'ปรับ' โดยใช้ ข้อมูลฝึก
-เมื่อพารามิเตอร์ของแบบจำลองถูกกำหนดแล้ว แบบจำลองสามารถใช้ทำการคาดการณ์กับข้อมูลใหม่ได้






